# **0: Summary**

Made by Leandro Veloso


leandrojpveloso@gmail.com
www.leandrojpveloso.com

[01-Tutorial transfer learning com ViT](https://theaisummer.com/hugging-face-vit/#training-the-model)


Kagle notebooks

[01:Covid19-detection-VIT and Heatmap](https://www.kaggle.com/code/leandrojpveloso/covid19-detection-with-vit-and-heatmap/edit)

[02:Eda](https://www.kaggle.com/code/leandrojpveloso/eda-vision-transformer-for-covid-19-detection/edit)

[03:Vit tutorial](https://www.kaggle.com/code/leandrojpveloso/vision-transformer-vit-tutorial-baseline/edit)

Git hubs

[01-Transformers tutorials](https://github.com/NielsRogge/Transformers-Tutorials/tree/master/VisionTransformer)

[02-Vit-pytorch](https://github.com/lucidrains/vit-pytorch)



# 1: Setting up Python

## 1.1: Basic package

In [ ]:
import matplotlib.pyplot as plt   # plotting
import numpy as np                # linear algebra
import pandas as pd               # data processing, CSV file I/O (e.g. pd.read_csv)
import os                         # accessing directory structure
import pickle                     # reading pickle files format
from google.colab import drive    # google drive conection
import seaborn as sns             # graph bar

In [ ]:
import pickle
from scipy import ndimage, misc
from   mpl_toolkits.mplot3d import Axes3D
from   glob import glob
from   tqdm import tqdm
import cv2 
import time
import sys, math

## 1.2: Tensorflow

In [ ]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow_addons as tfa

## 1.3: Keras

In [ ]:
import keras

## 1.4: Scikit packages

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

drive mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Alexandre's drive Path 
Alex_path = "/content/drive/MyDrive/00-desorganizado/Dataset"
Le_path   = "/content/drive/MyDrive/01-projetos_colab/03-PHd/01-Covid_lung_deep"

Mounted at /content/drive


## 1.5: Packages transformers

In [ ]:
!pip install tqdm
!pip install transformers datasets
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from datasets import Array3D, ClassLabel, Features, load_dataset
from matplotlib import pyplot
from numpy import inf
from sklearn.utils.class_weight import compute_class_weight
from torchinfo import summary
from tqdm import tqdm
from transformers import AdamW, ViTFeatureExtractor, ViTModel

In [ ]:
from datasets import Dataset, load_from_disk

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

<ipython-input-28-350fcae91df9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


# 2: Preprocessing data

In [ ]:
from torchvision import transforms

In [ ]:
# Code works using this
# load cifar10 
# train_ds, test_ds = load_dataset('cifar10', split=['train[:5000]', 'test[:2000]'])
# split up training into training + validation
# splits = train_ds.train_test_split(test_size=0.1)
# train_ds = splits['train']
# val_ds = splits['test'] 

In [ ]:
# load cifar10 
lung_dataset = load_from_disk(Le_path + '/01_data/data_set_lung')

print(lung_dataset)

/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


Dataset({
    features: ['img', 'label', 'fold'],
    num_rows: 193418
})


In [ ]:
fold_test = 1
fold_val  = 2
# Selecting every index different of validation and training indexes
train_index_true =  np.logical_not(
     np.isin(lung_dataset['fold'],[fold_test, fold_val])
  )

# Selecting every index different of validation and training indexes
val_index_true =  np.isin(lung_dataset['fold'],[fold_val])

# Getting indexes
N_img_total      = len(lung_dataset) 
val_index        = np.arange(N_img_total)[val_index_true]
train_index      = np.arange(N_img_total)[train_index_true]
 


[Dataset functions](https://huggingface.co/docs/datasets/v2.8.0/en/process)
As funcoes filter e select nos ajudam na selecao dos folds

In [ ]:
# split up training into training + validation
train_ds = lung_dataset.select(train_index[1:1000])
val_ds   = lung_dataset.select(val_index[1:200])

## 2.2: Feature extractor

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

In [ ]:
from PIL import Image
import numpy as np
import cv2

In [ ]:
chk = train_ds[67]
a = np.array(chk['img']) 
 
print(chk['label'])
pyplot.imshow(a, cmap=pyplot.get_cmap('gray'))

In [ ]:
chk['label']

In [ ]:
# Plot normal images
lista = [57,1242,91488,3121,613,123,5124,999,213]

# Build a 2 x 2 figure
rows, columns = 2, 3
fig, axes = plt.subplots(rows, columns) 

# Variable used for indending through the image list
current_index = 0

for i in range(rows):
    for j in range(columns):
        chk = train_ds[current_index]
        a = np.array(chk['img']) 

        axes[i, j].imshow(a)
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])
        axes[i, j].set_title(chk['label'])
        current_index +=1
        
plt.show()

In [ ]:
def preprocess_images(examples):
  
    # get batch of images
    images = examples['img']
    # convert to list of NumPy arrays of shape (C, H, W)
    images = [np.array(image, dtype=np.uint8) for image in images]
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    # preprocess and add pixel_values
    inputs = feature_extractor(images=images)
    examples['pixel_values'] = inputs['pixel_values']
    return examples

In [ ]:
imagem.shape

In [ ]:
examples= imagem
# get batch of images
images = examples['img']
# convert to list of NumPy arrays of shape (C, H, W)
images = [np.array(image, dtype=np.uint8) for image in images]
images = [np.moveaxis(image, source=-1, destination=0) for image in images]


In [ ]:
from datasets import Image,Array2D

In [ ]:
# we need to define the features ourselves as both the img and pixel_values have a 3D shape 
features = Features({
    'label': ClassLabel(names=['0:Normal','1:Pneumonia','2:Covid-19']),
    #'label': ClassLabel(names= classes),
    #'img': Array3D(dtype="int64", shape=(3,32,32)),
    'img': Image(decode=True, id=None),
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
})


In [ ]:
preprocessed_train_ds = train_ds.map(preprocess_images, batched=True, features=features)
preprocessed_val_ds = val_ds.map(preprocess_images, batched=True, features=features)
#preprocessed_test_ds = test_ds.map(preprocess_images, batched=True, features=features)

  0%|          | 0/1 [00:00<?, ?ba/s]

KeyError: ignored

In [ ]:
# get batch of images
images = val_ds['img']
# convert to list of NumPy arrays of shape (C, H, W)
images = [np.array(image, dtype=np.uint8) for image in images]
images = [np.moveaxis(image, source=-1, destination=0) for image in images]

# preprocess and add pixel_values
inputs = feature_extractor(images=images)
examples['pixel_values'] = inputs['pixel_values']
 

In [ ]:
# get batch of images
images = val_ds['img']
# convert to list of NumPy arrays of shape (C, H, W)
images = [np.array(image, dtype=np.uint8) for image in images]
images = [np.moveaxis(image, source=-1, destination=0) for image in images]


In [ ]:
np.expand_dims(chk, axis=3) 

In [ ]:
inputs = feature_extractor(images=images)

In [ ]:
train2=train_ds.remove_columns("fold")

In [ ]:
preprocessed_train_ds = train2.map(preprocess_images, batched=True, features=features)

In [ ]:
preprocessed_train_ds = train_ds.map(preprocess_images, batched=True, features=features)
preprocessed_val_ds   = val_ds.map(preprocess_images, batched=True, features=features)
#preprocessed_test_ds  = test_ds.map(preprocess_images, batched=True, features=features)

# Transformers - outra implementacao

In [ ]:
!pip install datasets -Uqq
!pip install transformers[sentencepiece] -Uqq
!pip install -qq wandb --upgrade


In [ ]:
from datasets import load_dataset


datasets = load_dataset('Matthijs/snacks')
labels = datasets['train'].features['label']


In [ ]:
labels

ClassLabel(names=['apple', 'banana', 'cake', 'candy', 'carrot', 'cookie', 'doughnut', 'grape', 'hot dog', 'ice cream', 'juice', 'muffin', 'orange', 'pineapple', 'popcorn', 'pretzel', 'salad', 'strawberry', 'waffle', 'watermelon'], id=None)

In [ ]:
from datasets import load_dataset


datasets = load_dataset('Matthijs/snacks')
labels = datasets['train'].features['label']


In [ ]:
from transformers import ViTFeatureExtractor


checkpoint = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(checkpoint)


In [ ]:
# data augmentation transformations
from torchvision.transforms import (
    Compose,
    Normalize,
    Resize,
    RandomResizedCrop,
    RandomHorizontalFlip,
    RandomAdjustSharpness,
    ToTensor,
    ToPILImage
)


# train
train_aug_transforms = Compose([
    RandomResizedCrop(size=feature_extractor.size),
    RandomHorizontalFlip(p=0.5),
    RandomAdjustSharpness(sharpness_factor=5, p=0.5),
    ToTensor(),
    Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])


# validation/test
valid_aug_transforms = Compose([
    Resize(size=(feature_extractor.size, feature_extractor.size)),
    ToTensor(),
    Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])


In [ ]:
def apply_train_aug_transforms(examples):
  examples['pixel_values'] = [train_aug_transforms(img.convert('RGB')) for img in examples['image']]
  return examples


def apply_valid_aug_transforms(examples):
  examples['pixel_values'] = [valid_aug_transforms(img.convert('RGB')) for img in examples['image']]
  return examples


datasets['train'].set_transform(apply_train_aug_transforms)
datasets['validation'].set_transform(apply_valid_aug_transforms)
datasets['test'].set_transform(apply_valid_aug_transforms)


In [ ]:
datasets['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 4838
})

In [ ]:
datasets_processed = datasets.rename_column('label', 'labels')